In [1]:
import os
from argparse import Namespace
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateLogger
from electra_spacing.train_lightning import KoELECTRASpacing

In [2]:
lr_logger = LearningRateLogger()

parser = {}
parser['gpus'] = 1

hparams = {}
hparams['file_path'] = './base_text.tsv'
hparams["max_epochs"] = 10
hparams['train_ratio'] = 0.7
hparams['batch_size'] = 64
hparams['weight_value'] = 10
hparams['learning_rate'] = 0.001
hparams['warmup_ratio'] = 0.2

parser = Namespace(**parser)
hparams = Namespace(**hparams)
callbacks=[lr_logger],
trainer = Trainer.from_argparse_args(parser, callbacks=[lr_logger] )
# trainer = Trainer(max_epochs = parser["max_epochs"], gpu=parser['gpus'])

model = KoELECTRASpacing(hparams)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                  | Params
--------------------------------------------------
0 | model   | KoELECTRASpacingModel | 21 M  
1 | loss_fn | CrossEntropyLoss      | 0     
/home/svcapp/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/svcapp/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/svcapp/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [3]:
##inter test

In [2]:
model = KoELECTRASpacing.load_from_checkpoint('./lightning_logs/version_0/checkpoints/epoch=2.ckpt')

In [44]:
text = "로밍해지하고싶음"

In [45]:
from electra_spacing.tokenizer import tokenize
from electra_spacing.tokenizer import get_tokenizer
import numpy as np

In [46]:
tokens, segment_ids = tokenize(text, seq_len=128)

In [47]:
segment_ids.shape

torch.Size([128])

In [48]:
pred = model.model.forward(tokens.unsqueeze(0), segment_ids.unsqueeze(0))

In [49]:
pred_tokens = pred.argmax(-1).squeeze().numpy()

In [50]:
end_idx = np.where(pred_tokens == 3)[0][0]

In [51]:
end_idx

11

In [52]:
tokenizer = get_tokenizer()

In [53]:
pred_tokens[:end_idx]

array([    2, 25831,  6363,  6825,  6825,   617, 18529,     2,   228,
        1280,    35])

In [54]:
tokenizer.decode(pred_tokens[:end_idx], skip_special_tokens=True)

'로밍 해지 하고 하고 싶 알고 물 좀 해'